In [ ]:
import numpy as np
import numba
from numba import jit , njit

In [ ]:
# Global Config Variables
n0 = 1000  # number of p=0 points in metric space
V = n0  # Threshold for p=0
K = 10# No of clusters
A= 2# No of attributes
iterations = 1  # maximum iteration in clustering
runs =10
option='Kmeans'  #Kmedian
dataset ='Diabetes'  #CensusII

In [ ]:
import numpy as np
import os
import sys
import pandas
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import log2
from sklearn import preprocessing
import sys
import timeit


In [ ]:
import datetime
import json
import random
from collections import defaultdict
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import cm
#from pyclustering.cluster.kmedians import kmedians
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import os


In [ ]:
from scipy.spatial import distance


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.cluster import KMeans
!pip install scikit-learn-extra
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
from sklearn.metrics import pairwise_distances



In [ ]:
print(os.getcwd())
def load_CensusII(data_dir=''):

    data_dir = data_dir
    _path = 'USCensus1990_p.csv'
    data_path = os.path.join(data_dir, _path)

    K = 10

    df = pandas.read_csv(data_path, sep=',')

    
    return df
#load_CensusII()

In [ ]:
print(os.getcwd())
def load_Dia(data_dir=''):

    data_dir = data_dir
    _path = 'diabetic_data_p.csv'
    data_path = os.path.join(data_dir, _path)

    K = 10

    df = pandas.read_csv(data_path, sep=',')
    print(df.head())
    print(len(df))
    
    return df
load_Dia()

/home/ram/Downloads
        age  time_in_hospital  type
0 -0.958493         -0.285117     1
1 -0.989471         -0.144732     1
2 -0.954526         -0.298129     1
3  0.382705          0.923871     1
4  0.581613          0.813466     1
101763


,age,time_in_hospital,type
0,-0.958493,-0.285117,1
1,-0.989471,-0.144732,1
2,-0.954526,-0.298129,1
3,0.382705,0.923871,1
4,0.581613,0.813466,1
...,...,...,...
101758,-0.415145,0.909755,0
101759,0.723974,-0.689828,0
101760,0.771261,-0.636519,0
101761,0.445848,-0.895109,0


In [ ]:

if dataset=='CensusII':
    df=load_CensusII()
    df= df.round(decimals=5)
    print(len(df))
    df = df.dropna()
    print(len(df))
    #df['type'] = df['type']-1
    typ = df['type'].values
    #print(len(typ))
    #print(df.head(10))
    c1 = np.count_nonzero(typ == 0)
    c2 = np.count_nonzero(typ == 1)

    print(c1/(c1+c2))
    print(c2/(c1+c2))

    print(c1)
    print(c2)
    dfDropped = df.drop(columns=['type'])

elif dataset=='Diabetes':
    df=load_Dia()
    df= df.round(decimals=5)
    print(len(df))
    df = df.dropna()
    #df = drop_duplicates()
    print(len(df))
    #df['type'] = df['type']-1
    typ = df['type'].values
    print(len(typ))
    #print(df.head(10))
    c1 = np.count_nonzero(typ == 0)
    c2 = np.count_nonzero(typ == 1)

    print(c1/(c1+c2))
    print(c2/(c1+c2))

    print(c1)
    print(c2)

    dfDropped = df.drop(columns=['type'])
    
    

        age  time_in_hospital  type
0 -0.958493         -0.285117     1
1 -0.989471         -0.144732     1
2 -0.954526         -0.298129     1
3  0.382705          0.923871     1
4  0.581613          0.813466     1
101763
101763
101763
101763
0.46239792458948736
0.5376020754105126
47055
54708


In [ ]:
def VanillaKmeans(X, k,seedValue):
    """
    specify the number of clusters k and
    the maximum iteration to run the algorithm
    """
    n_row, n_col = X.shape
    maxiter=100
    
    # randomly choose k data points as initial centroids
    
    
    rand_indices = np.random.choice(n_row, size = k)
    centroids = X[rand_indices]
    cost_variation=[]
    cnt =0
    for itr in range(maxiter):
        # compute distances between each data point and the set of centroids
        # and assign each data point to the closest centroid
        distances_to_centroids = distance.cdist(X, centroids, 'sqeuclidean')
        cluster_assignment = np.argmin(distances_to_centroids, axis = 1)

        # select all data points that belong to cluster i and compute
        # the mean of these data points (each feature individually)
        # this will be our new cluster centroids
        new_centroids=[]
        for i  in range(k):
            assign= np.array([X[cluster_assignment == i]])
            med = np.mean(assign[0],axis=0)
            new_centroids.append(med)             
        
        # if the updated centroid is still the same,
        # then the algorithm converged
        new_centroids = np.array(new_centroids)
        
        if np.all(centroids == new_centroids):
                break
        
        centroids = new_centroids
        
        heterogeneity = 0
        for i in range(k):
            # note that pairwise_distance only accepts 2d-array
            cluster_data = X[cluster_assignment == i]
            distances = distance.cdist(cluster_data, [centroids[i]],'euclidean')
            heterogeneity += np.sum(distances ** 2) #sq euclidean 
        cost_variation.append(heterogeneity)
        
    
    return centroids, cluster_assignment, heterogeneity,cost_variation#(cost)

def VanillaKmedian(X, k,seedValue):
    """
    specify the number of clusters k and
    the maximum iteration to run the algorithm
    """
    n_row, n_col = X.shape
    maxiter=100
    
    # randomly choose k data points as initial centroids
    #if seed is not None:
     #   np.random.seed(seed)
    
    rand_indices = np.random.choice(n_row, size = k)
    centroids = X[rand_indices]
    cost_variation=[]
    cnt =0
    for itr in range(maxiter):
        # compute distances between each data point and the set of centroids
        # and assign each data point to the closest centroid
        distances_to_centroids = pairwise_distances(X, centroids, metric = 'sqeuclidean',force_all_finite=True)
        cluster_assignment = np.argmin(distances_to_centroids, axis = 1)

        # select all data points that belong to cluster i and compute
        # the mean of these data points (each feature individually)
        # this will be our new cluster centroids
        new_centroids=[]
        for i  in range(k):
            #print(i)
            assign= np.array([X[cluster_assignment == i]])
            med = np.median(assign[0],axis=0)
            new_centroids.append(med)             
       
        # if the updated centroid is still the same,
        # then the algorithm converged
        new_centroids = np.array(new_centroids)
       
        if np.all(centroids == new_centroids):
    
                break
        
        centroids = new_centroids
        
        heterogeneity = 0
        for i in range(k):
            # note that pairwise_distance only accepts 2d-array
            cluster_data = X[cluster_assignment == i]
            distances = pairwise_distances(cluster_data, [centroids[i]], metric = 'euclidean')
            heterogeneity += np.sum(distances ** 2) #sq euclidean 
        cost_variation.append(heterogeneity)
        
    
    return centroids, cluster_assignment, heterogeneity,cost_variation#(cost)

In [ ]:
def dual_print(f,*args,**kwargs):
    #print(*args,**kwargs)
    print(*args,**kwargs,file=f)

def load_dataset(csv_name):
    # read the dataset from csv_name and return as pandas dataframe
    df = pd.read_csv(csv_name, header=None)
    return df


def k_random_index(df,K):
    # return k random indexes in range of dataframe
    return random.sample(range(0, len(df)), K)

def find_k_initial_centroid_Kmeans(df,K,seedValue):
    kmeans=KMeans(n_clusters=int(K),random_state=int(seedValue)).fit(df)
    return kmeans.cluster_centers_

          

def find_k_initial_centroid(df,K):
    centroids = []    # make of form [ [x1,y1]....]

    rnd_idx = k_random_index(df,K)
    #print(rnd_idx)
    for i in rnd_idx:
        coordinates =[]
        for a in range(0,A):
            coordinates.append(df.loc[i][a])
        centroids.append(coordinates)   #df is X,Y,....., Type

    return centroids

#nOt using
def calc_distance(x1, y1, x2, y2):
    # returns the euclidean distance between two points
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def calc_distance_a(centroid, point):
    #print('çalculating distance\n')

    sum_ = 0

    for i in range(0, len(centroid)):
        sum_ = sum_ + (centroid[i]-point[i])**2

    return sum_ #**0.5

@njit(parallel=False)
def find_distances_fast(k_centroids, df):
   
    dist = np.zeros((len(k_centroids),len(df),A+2),np.float64)
    Kcnt = 0 
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
       
        l = np.zeros((len(df),A+2),np.float64)
        
       
        index = 0 
        for row in df:                # row is now x,y,z......type
            # append all coordinates to point
            dis = np.sum((c- row[:A])**2)#calc_distance_a(c, point)
            #Processing the vector for list
            row_list = np.array([dis])
            #append distance or l norm
            row_list = np.append(row_list,row[:A+1])
            #append all coordinates #append type of this row
  
            l[index] = row_list
            index = index + 1
            #l.append([calc_distance(c[0], c[1], row[0], row[1]), row[0], row[1], row[2]])  # [dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist[Kcnt]= l
        Kcnt = Kcnt + 1

    # return dist which contains distances of all points from every centroid

    return dist

def find_distances(k_centroids, df):
    dist = []
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
        l = []
       
        for index, row in df.iterrows():                # row is now x,y,z......type
            point =[]
            for a in range(0, A):
                point.append(row.iloc[a])  # append all coordinates

            dis = calc_distance_a(c, point)
            #Processing the vector for list
            row_list = [dis]
            #append distance or l norm

            for a in range(0, A):
                row_list.append(row.iloc[a])   #append all coordinates
            #print(row.iloc[a+1])
            row_list.append(row.iloc[a+1])   #append type of this row

            l.append(row_list)
            #l.append([calc_distance(c[0], c[1], row[0], row[1]), row[0], row[1], row[2]])  # [dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist.append(l)

    # return dist which contains distances of all points from every centroid

    return dist


def sort_and_valuation(dist):
    sorted_val = []

    for each_centroid_list in dist:
        each_centroid_list_sorted = sorted(each_centroid_list, key=lambda x: (x[A+1], x[0]))  # A+1 is index of type , 0 is dist
        sorted_val.append(each_centroid_list_sorted)

        # sort on basis of type & then dist.
        # Now all whites are towards start and all black are after white as they have additional V added to their valuation
        # Among the whites, the most closest is at start of list as it has more valuation.
        # Similarly sort the black points among them based on distance as did with white

    return sorted_val

def clustering(sorted_valuation, hashmap_points,K):
    n = len(sorted_valuation[0])  # total number of points in metric space
    
    cluster_assign = []

    for i in range(0, K):
        cluster_assign.append([])  # initially all clusters are empty
    
    map_index_cluster = []
    for i in range(0,K+2):
        map_index_cluster.append(0)
        #initially check all sorted evaluation from 0th index 
    
    number_of_point_alloc = 0
    curr_cluster = 0
    
    # until all points are allocated
    while number_of_point_alloc != n:  # As convergence is guaranteed that all points will be allocated to some cluster set
       
        start_inde = map_index_cluster[curr_cluster % K]
        
        for inde in range(start_inde,len(sorted_valuation[curr_cluster % K])):
            each = sorted_valuation[curr_cluster % K][inde]
            # each is (dist,X,Y,....Z,type)
            
            
                    
            if tuple(each[1:]) in hashmap_points.keys():# each is (dist, X,Y,....Z, type)
           
             
                if int(hashmap_points[tuple(each[1: ])]) > 0:    #if some points of that type left to allocate
                    cluster_assign[curr_cluster].append(each)
                    hashmap_points[tuple(each[1: ])] -= 1
                    number_of_point_alloc += 1
                   
                    map_index_cluster[curr_cluster % K] = inde  #next time start from here as isse prev all allocated
                    break

              
        curr_cluster = (curr_cluster + 1) % K
    
    return cluster_assign



def update_centroids_median(cluster_assign,K):
    new_centroids = []
    for k in range(0, K):
      
        cAk =  np.array(cluster_assign[k])
        cAk = np.delete(cAk,[0,-1],axis=1)
        #print(len(cAk))
        if len(cAk) %2 ==0 and len(cAk)>0: 
            cc = [np.median(np.array(cAk[:-1])[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk) %2 !=0 and len(cAk)>0:
            cc = [np.median(np.array(cAk)[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk)==0:
            print("Error: No centroid found updation error")
  
    return new_centroids
        


def update_centroids(cluster_assign,K):

    new_centroids = []
    for k in range(0, K):

        sum_a = []

        for i in range(0, A):
            sum_a.append(0)

        for each in cluster_assign[k]:
            sum_a = [sum(x) for x in zip(sum_a, each[1:-1])]
            #each is (dist,X,Y,.....Z,type)
       # print('sum a is '+str(sum_a))
        new_coordinates = []
        for a in range(0, A):
            new_coordinates.append(sum_a[a] / len(cluster_assign[k]))
        new_centroids.append(new_coordinates)
        k=k+1



    return new_centroids


def calc_clustering_objective(k_centroid, cluster_assign,K):
    cost = 0

    for k in range(0, K):

        for each in cluster_assign[k]:  #each is (dist, X,Y,....,Z,type)
            dd = calc_distance_a(k_centroid[k], each[1:-1])
            cost = cost + (dd)
#            print('K: '+ str(k)+ 'Each '+ str(each[1:-1])+ ' centroid ; '+ str(k_centroid)+ 'dist : '+ str(dd))

    return cost


def calc_fairness_error(df, cluster_assign,K):
    U = []  # distribution of each type in original target dataset for each J = 0 , 1....
    P_k_sum_over_j = []  # distribution in kth cluster  sum_k( sum_j(   Uj * j wale/total_in_cluster ) )

    f_error = 0
    cnt_j_0 = 0
    cnt_j_1 = 0
  #  cnt_j_2 = 0
    cnt = 0
    for index, row in df.iterrows():
        if row.iloc[-1] == 1:
            cnt_j_1 += 1
        elif row.iloc[-1] == 0:
            cnt_j_0 += 1
      #  elif row.iloc[-1] == 2:
        #    cnt_j_2 += 1
            
        cnt += 1

    U.append(cnt_j_0 / cnt)
    U.append(cnt_j_1 / cnt)
    #U.append(cnt_j_2 / cnt)

   
    for k in range(0, K):  # for each cluster

        for j in range(0, len(U)):   #for each demographic group

            cnt_j_cluster = 0
            cnt_total = 0

            for each in cluster_assign[k]:
                if int(each[-1]) == j:    #each is (dist,X, Y.....,Z,type)
                    cnt_j_cluster += 1
                cnt_total += 1
                
            if cnt_j_cluster !=0 and cnt_total != 0:
                P_k_sum_over_j.append(-U[j] * np.log((cnt_j_cluster / cnt_total)/U[j]))
            else:
                P_k_sum_over_j.append(0)  #log(0)=0 considered

    for each in P_k_sum_over_j:
        f_error += each

    return f_error


def calc_balance(cluster_assign,K):
    S_k = []  # balance of each k cluster
    balance = 0  # min (S_k)

    for k in range(0, K):
        cnt_j_0 = 0
        cnt_j_1 = 0
       # cnt_j_2 = 0
        cnt = 0
        for each in cluster_assign[k]:

            if int(each[-1]) == 1:
                cnt_j_1 += 1
            elif int(each[-1]) == 0:
                cnt_j_0 += 1
           # elif int(each[-1]) == 2:
           #     cnt_j_2 += 1
                
            cnt += 1

        if cnt_j_0 != 0 and cnt_j_1 != 0 :#and cnt_j_2!= 0:
            S_k.append(min([cnt_j_0 / cnt_j_1, cnt_j_1 / cnt_j_0 ]))#, cnt_j_1 / cnt_j_2 , cnt_j_2 / cnt_j_1 , cnt_j_0 / cnt_j_2, cnt_j_2 / cnt_j_0 ]))
        elif cnt_j_0 == 0 or cnt_j_1 ==0  :#or cnt_j_2==0:
            S_k.append(0)



    balance = min(S_k)

    return balance


In [ ]:


def main():
    # Step1 : Load the dataset
            
    list_fair_K=[]
    list_obj_K =[]       
    list_balance_K=[]
    
    os.makedirs('Diabetes_kmeans_center')
    
    for kk in [2,5,10,15,20,30,40]:
        K = kk
        
        print(" K=="+str(K)+"  ")
        
        list_fair_run=[]
        cost_variation_ck_collec =[]
        cost_variation_collec = []
        list_obj_run =[]       
        list_balance_run=[]
        seeds = [0,100,200,300,400,500,600,700,800,900,1000,1100]
      
        for run in range(0,runs):
            np.random.seed(seeds[run])
            random.seed(seeds[run])
            f = open('Diabetes_kmeans_center/K_'+str(K)+'_run_'+str(run)+'_output.txt', 'a')
          

            list_fair_iter=[]
            list_obj_iter =[]
            list_balance_iter=[]

           
            # Step2 : Find initial K random centroids using k_random_index(df) & find_k_initial_centroid(df)
            if option=='Kmeans':
                k_centroid,_,_,cost_variation= VanillaKmeans(dfDropped.values,kk,seeds[run])#VanillaKmeans()find_k_initial_centroid_Kmeans(dfDropped,kk,seeds[run])#find_k_initial_centroid(df,kk)
            else:
                k_centroid,_,_,cost_variation= VanillaKmedian(dfDropped.values,kk,seeds[run])#VanillaKmeans()find_k_initial_centroid_Kmeans(dfDropped,kk,seeds[run])#find_k_initial_centroid(df,kk)
                
            cost_variation_ck = []
            
            for eac in cost_variation:
                cost_variation_ck.append(eac)
            
           
            prev_assignment =[]
            cluster_assignment = []

            for i in range(0, K):
                cluster_assignment.append([])  # initially all clusters are empty

            sum_time = 0
            curr_itr = 0
            prev_objective_cost=-1
            objective_cost = 0
                # Step3 : Find distances from the centroids using find_distances() with list of [ [x1,y1,z1..] , [x2,y2,z2..]....] centroids format list
            while curr_itr < iterations:# and prev_objective_cost != objective_cost:

                start = time.process_time()#timeit.default_timer()
             
                df1 = df.values
                k_centroids1= np.array(k_centroid)
                

                dist = find_distances_fast(k_centroids1, df1)
               
                valuation = sort_and_valuation(dist)
               
                hash_map = {}
            
                for index, row in df.iterrows():
                    temp = tuple(row[:-1])
                    
                    if tuple(row) in hash_map.keys():
                    
                        hash_map[tuple(row)] = int(hash_map[tuple(row)]) + 1
                
                    else:
                        
                        hash_map.update({tuple(row): int(1)})   #dict is of form { (x,y): 0 , ....}
            
                prev_assignment = cluster_assignment
                cluster_assignment = clustering(valuation, hash_map,K)

                
                balance = calc_balance(cluster_assignment,K)
               
                f_error = calc_fairness_error(df, cluster_assignment,K)
              
                clustering_cost = calc_clustering_objective(k_centroid,cluster_assignment,K)
                if curr_itr!=0:
                     prev_objective_cost = objective_cost
                    
                objective_cost = np.round(clustering_cost,3)
                
                cost_variation_ck.append(objective_cost)
                

                list_balance_iter.append(str(balance))
                list_obj_iter.append(str(objective_cost))
                list_fair_iter.append(str(f_error))

               

                #Step7 : Find new centroids using mean of all points in current assignment
                if option=='Kmeans':
                    k_centroid_temp = update_centroids(cluster_assignment,K)
                else:
                    k_centroid_temp = update_centroids_median(cluster_assignment,K)
                    
                clustering_cost_temp = calc_clustering_objective(k_centroid_temp,cluster_assignment,K)
                cost_variation.append(np.round(clustering_cost_temp,3))
               
                #Step8 : Repeat from Step3 until clusters are same or iterations reach upper limit
                stop = time.process_time()#timeit.default_timer()
                sum_time += (stop - start)
               
                curr_itr += 1

                dual_print(f,'-----------------------------Finished-----------------------------------------------\n')


            #Step 10 : Find balance , fairness error , and clustering objective or cost

            balance_converged = calc_balance(cluster_assignment,K)
            f_error_converged = calc_fairness_error(df, cluster_assignment,K)
            clustering_cost_converged = calc_clustering_objective(k_centroid,cluster_assignment,K)

         
            cost_variation_collec.append(cost_variation)
           
            cost_variation_ck_collec.append(cost_variation_ck)
            
            f.close()
            run  = run +1
            list_obj_run.append(clustering_cost_converged)
            list_fair_run.append(f_error_converged)
            list_balance_run.append(balance_converged)
        

        print("Cost variations over run\n")
        print(str(cost_variation_collec))
        #print("\nCost variations over run- CK\n")
        #print(str(cost_variation_ck_collec))
        print("\nbalance variations over run")
        print(str(list_balance_run))
        print("\nfairness error over run")
        print(str(list_fair_run))
        print("#"*30)
        
        
        list_obj_K.append(np.mean(np.array(list_obj_run)))
        list_fair_K.append(np.mean(np.array(list_fair_run)))
        list_balance_K.append(np.mean(np.array(list_balance_run)))
        
   

In [ ]:

import time
if __name__ == "__main__":
    main()
    